# Indonesia Stamp Detection

This project goals is detecting the collectible stamp based on the value of the stamp, year of released, and the series. This project uses Yolov5 as the detection model and Roboflow to annotate the dataset.

Clone YoloV5 Repository and install its depedencies.
Also, install Roboflow Library via pip to be able to clone dataset that has been created using Roboflow

In [ ]:
#clone YOLOv5 and
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt # install dependencies
%pip install -q roboflow

Cloning into 'yolov5'...
remote: Enumerating objects: 16098, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 16098 (delta 15), reused 14 (delta 2), pack-reused 16056
Receiving objects: 100% (16098/16098), 14.65 MiB | 4.55 MiB/s, done.
Resolving deltas: 100% (11047/11047), done.
/content/yolov5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 660.4/660.4 kB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
imageio 2.31.6 requires pillow<10.1.0,>=8.3.2, but you have pillow 10.1.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Import following library and get PyTorch version with GPU avaibility on the hosted runtime. Make sure you have changed the runtime to GPU by changing runtime to GPU.

Changing the runtime:
1. Click the dropdown beside the computer resource stat
2. CLick "Change Runtime"
3. Select the T4 GPU
4. Click "Save"

In [ ]:
import torch
import os
from IPython.display import Image, clear_output, display  # to display images
import gc
import glob

from roboflow import Roboflow

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Setup complete. Using torch 2.1.0+cu118 (Tesla T4)


Clone the dataset we've created on the Roboflow by clicking "Export Dataset" and selecting "Show Dowload Code". Paste the code to the column below

In [ ]:
rf = Roboflow(api_key="DOK91JNrPu8ft2QBccOo")
project = rf.workspace("ape").project("indonesia-stamp-recognition")
dataset = project.version(1).download("yolov5")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Indonesia-Stamp-Recognition-1 in yolov5pytorch:: 100%|██████████| 654/654 [00:11<00:00, 55.97it/s] 


Clear the cache on the memory of the hosted runtime and set the training split data size

In [ ]:
gc.collect()

torch.cuda.empty_cache()
gc.collect()

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:1024"

Set the image size, Bigger image can improve the accuracy but slowing the training process. Determine the batch size to speed the training process and generization of the dataset.

In [ ]:
!python train.py --img 960 --batch 16 --epochs 250 --data "Indonesia-Stamp-Recognition-1/data.yaml" --weights yolov5s.pt --cache

2023-12-11 08:07:38.966017: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-11 08:07:38.966078: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-11 08:07:38.966121: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5s.pt, cfg=, data=Indonesia-Stamp-Recognition-1/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=250, batch_size=16, imgsz=960, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_b

In [ ]:
os.environ["DATASET_DIRECTORY"] = "/content/yolov5"
!python detect.py --weights "runs/train/exp4/weights/best.pt" --img 480 --conf 0.5 --source "Indonesia-Stamp-Recognition-1/test/images"

detect: weights=['runs/train/exp2/weights/best.pt'], source=Indonesia-Stamp-Recognition-1/test/images, data=data/coco128.yaml, imgsz=[480, 480], conf_thres=0.3, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-247-g3f02fde Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7134187 parameters, 0 gradients, 16.1 GFLOPs
image 1/13 /content/yolov5/Indonesia-Stamp-Recognition-1/test/images/1-5_jpg.rf.4406a1c38f69cf027df7382d175ea7c9.jpg: 384x480 1 1000-Layang_Layang-2002, 1 500-ITIK_ALIS-1998, 1 700-SERANGAN_UMUM_1_MARET_1949-1996, 1 75-PELITA_V-1991, 69.5ms
image 2/13 /content/yolov5/Indonesia-Stamp-Recognition-1/test/image

In [ ]:
#display inference on ALL test images
for imageName in glob.glob('/content/yolov5/runs/detect/exp2/*.jpg'): #assuming JPG
    display(Image(filename=imageName))
    print("\n")

Export the model to the device

In [ ]:
!zip -r "model.zip" "runs/train/exp2"

  adding: runs/train/exp2/ (stored 0%)
  adding: runs/train/exp2/opt.yaml (deflated 48%)
  adding: runs/train/exp2/results.png (deflated 8%)
  adding: runs/train/exp2/train_batch0.jpg (deflated 3%)
  adding: runs/train/exp2/P_curve.png (deflated 10%)
  adding: runs/train/exp2/val_batch0_pred.jpg (deflated 9%)
  adding: runs/train/exp2/weights/ (stored 0%)
  adding: runs/train/exp2/weights/best.pt (deflated 10%)
  adding: runs/train/exp2/weights/last.pt (deflated 10%)
  adding: runs/train/exp2/confusion_matrix.png (deflated 14%)
  adding: runs/train/exp2/labels.jpg (deflated 38%)
  adding: runs/train/exp2/train_batch2.jpg (deflated 3%)
  adding: runs/train/exp2/events.out.tfevents.1702282061.fde822f14aad.2551.0 (deflated 23%)
  adding: runs/train/exp2/results.csv (deflated 84%)
  adding: runs/train/exp2/R_curve.png (deflated 10%)
  adding: runs/train/exp2/train_batch1.jpg (deflated 3%)
  adding: runs/train/exp2/labels_correlogram.jpg (deflated 42%)
  adding: runs/train/exp2/F1_curve.png

Dowload Tested Images

In [ ]:
!zip -r "detect.zip" "runs/detect/exp2"

updating: runs/detect/exp2/ (stored 0%)
  adding: runs/detect/exp2/IMG_20231129_102626_jpg.rf.30c993ab201ba24a8edae4e1ba14c2a6.jpg (deflated 11%)
  adding: runs/detect/exp2/IMG_20231122_141126_jpg.rf.db204e5a692325096a01e544f42df78a.jpg (deflated 8%)
  adding: runs/detect/exp2/IMG_20231122_141151_jpg.rf.c6d59701bef8d2299c6e3dad0dc78485.jpg (deflated 8%)
  adding: runs/detect/exp2/IMG_20231129_102043_jpg.rf.ceece35f9fbfbdb8c662605954dc23e4.jpg (deflated 11%)
  adding: runs/detect/exp2/3-6_jpg.rf.6faaacbfba39469dc832b9c727a28f4d.jpg (deflated 8%)
  adding: runs/detect/exp2/4-1_jpg.rf.2a41e438e41d8a2abc60543c13b8b08d.jpg (deflated 6%)
  adding: runs/detect/exp2/1-8_jpg.rf.cddd45dd940b6fffadbd1dccaa3120f7.jpg (deflated 7%)
  adding: runs/detect/exp2/IMG_20231122_141426_jpg.rf.db0fa5ff371f3cba76eeb7de0516a658.jpg (deflated 8%)
  adding: runs/detect/exp2/IMG_20231122_141426_jpg.rf.a8adb85bce737db298cdaf03c2e23521.jpg (deflated 8%)
  adding: runs/detect/exp2/IMG_20231122_141151_jpg.rf.ddd9f65